## 데이터 로드

In [54]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [55]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [56]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [57]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [58]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [59]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [60]:
train_data_sample = train_data[:500]

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [61]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [62]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'걸그룹댄스', '여자아이돌', 'kpop', '댄스', '스트레스해소'}


In [63]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(16674, 2)

In [64]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [65]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [66]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16674 entries, 0 to 16673
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                16674 non-null  object
 1   song_id             16674 non-null  int64 
 2   gnr                 16674 non-null  object
 3   issue_date          16674 non-null  int64 
 4   album_id            16674 non-null  int64 
 5   artist_id_basket    16674 non-null  object
 6   song_name           16674 non-null  object
 7   artist_name_basket  16674 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.1+ MB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [67]:
train_data_sample2 = train_data[:500]

In [89]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 1, workers = 4, sg = 1)

w2v.wv.vectors.shape

(727, 100)

In [90]:
print(w2v.wv.most_similar('스트레스'))

[('태교클래식', 0.34352555871009827), ('최신', 0.30779463052749634), ('레깅스', 0.28482699394226074), ('파티', 0.26874759793281555), ('거슈윈', 0.2662806808948517), ('런쥔', 0.2657091021537781), ('연휴', 0.2536941170692444), ('잔잔한', 0.25109046697616577), ('중음', 0.25108835101127625), ('감성', 0.24760691821575165)]


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(16674, 194)

In [92]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.70710678,
        0.70710678],
       [0.        , 0.        , 0.        , ..., 0.70710678, 1.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.5       ,
        1.        ]])

In [93]:
simi_test = gnr_sim[1, :]
simi_test2 = gnr_sim[2, :]
print(simi_test + simi_test2)
print(song_data[song_data['song_id']==525514])
print(song_data[song_data['song_id']==129701])
print(song_data[song_data['song_id']==229622])

[0.         1.33333333 1.33333333 ... 0.         0.         0.        ]
                     gnr  issue_date  album_id artist_id_basket  \
525514  [GN1402, GN1401]    20130506   2200223         [734201]   

              song_name artist_name_basket  song_id  
525514  Hey Little Girl          [The Sol]   525514  
                             gnr  issue_date  album_id artist_id_basket  \
129701  [GN0901, GN0902, GN1001]    20130917   2201802         [536907]   

       song_name artist_name_basket  song_id  
129701   Octagon      [Royal Bangs]   129701  
                                     gnr  issue_date  album_id  \
229622  [GN1402, GN1401, GN0901, GN0902]    20061212    528351   

       artist_id_basket  song_name artist_name_basket  song_id  
229622         [406781]  Say Hello     [Rosie Thomas]   229622  


- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [120]:
import numpy as np

def find_sim_song(df, sim_matrix, songs, top_n=10):
    simi = np.zeros(len(df['song_id']))
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        simi = simi + sim_matrix[title_index, :]
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    
    #for song in songs:
    #    title_song = df[df['song_id'] == song]
    #    title_index = title_song.index.values
    #    
    #    temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
    
    # 유사도가 0.5 이하인 경우는 제외
    #temp = temp[temp['similarity'] >= 0.5]
    
    temp = temp.reset_index(drop=True)
    
    #final_index = temp.index.values[ : top_n]
    
    return temp.iloc[ : top_n]

In [121]:
similar_songs = find_sim_song(song_tag_appended, gnr_sim, [525514, 129701, 229622], 10)

similar_songs[['song_id', 'similarity', 'issue_date', 'gnr']]

,song_id,similarity,issue_date,gnr
0,191430,0.761486,20140407,"[GN1402, GN1401, GN0901, GN0902]"
1,463782,0.761486,20170227,"[GN1402, GN1401, GN0901, GN0902]"
2,229622,0.761486,20061212,"[GN1402, GN1401, GN0901, GN0902]"
3,205238,0.606558,20110124,"[GN0904, GN1402, GN1401, GN0901, GN1001]"
4,701978,0.575727,20101011,"[GN1402, GN1401, GN1001]"
5,513731,0.569036,20111018,"[GN1402, GN1401]"
6,489449,0.569036,20111111,"[GN1402, GN1401]"
7,392798,0.569036,20111018,"[GN1402, GN1401]"
8,525514,0.569036,20130506,"[GN1402, GN1401]"
9,598147,0.569036,20131105,"[GN1402, GN1401]"


## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [96]:
def song_recommend(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
        print(simi_songs)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 태그와 유사도 두 부분에서 동일한 개수(홀수일 경우 태그 > 유사도) 추출
        # sc = 유사도 부분에서 추출할 노래의 개수
        if len(recommended) % 2 == 0:
            sc = (10-len(recommended)) / 2
        else:
            sc = (10-len(recommended)) // 2
        
        # 태그는 있고 히스토리가 없는 경우 태그 부분에서 10개를 추출하기 위한 코드
        if len(songs) == 0:
            sc = 0
        
        # 이미 추출한 노래를 제외하고 태그 부분에서 정해진 개수만큼 추출한다
        # 태그가 없을 경우 동작하지 않음
        index = 0
        while len(tag_songs) != 0 and len(recommended) < (10 - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommended:
                recommended.append(tag_song)
            
            index += 1
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]

In [104]:
def song_recommend_test(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
    
    
    
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10 and len(simi_songs) != 0 and len(simi_songs) > index:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
        if len(recommended) < 10:
            index = 0
            while len(tag_songs) != 0 and len(recommended) < 10:
                tag_song = tag_songs[index][0]
                if not tag_song in recommended:
                    recommended.append(tag_song)
                index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]

### 태그와 노래 목록 존재

In [98]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

                                             tags  song_id  \
0                                   [휴식, 힐링, 잔잔한]   191430   
1      [감성적인, 휴가, 주말, 분위기, 휴식, 책읽을때, 추억, 혼자, 잔잔한]   463782   
2                                    [사랑, 겨울, 설렘]   229622   
3                                             [락]   205238   
4                                     [봄, 힐링, 카페]   701978   
..                                            ...      ...   
95                                 [매장음악, 팝, 잔잔한]   315691   
96                                   [드라이브, 스트레스]   458897   
97                                     [스트레스, 운동]   486784   
98                                   [드라이브, 스트레스]   231210   
99  [행복, 집, 꿈, 산책, 매장음악, 드라이브, 바람, 몽환, 가을감성, 잔잔한]   603631   

                                         gnr  issue_date  album_id  \
0           [GN1402, GN1401, GN0901, GN0902]    20140407   2249887   
1           [GN1402, GN1401, GN0901, GN0902]    20170227  10041704   
2           [GN1402, GN1401, GN0901, GN0902] 

,song_id,song_name,artist_name_basket,issue_date,tags
658,562575,Lucky Strike,[Maroon 5],20120625,"[드라이브, 주말, 느껴져, 날려버려, 봄, 운동, 여행, 기분전환, 댄스, 파티,..."
648,259807,Marry You,[Bruno Mars],20101005,[락]
656,291570,Grenade,[Bruno Mars],20101206,"[팝, 락]"
677,574134,The Lazy Song,[Bruno Mars],20130401,[락]
3597,505710,The Scientist,[Coldplay],20020901,"[Coldplay, Greenday, 기분전환, 비오는날, 휴식, 힐링, 빗물같은,..."
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 힐링, 잔잔한]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[감성적인, 휴가, 주말, 분위기, 휴식, 책읽을때, 추억, 혼자, 잔잔한]"
4912,229622,Say Hello,[Rosie Thomas],20061212,"[사랑, 겨울, 설렘]"
9,205238,"Dingue, Dingue, Dingue",[Christophe Mae],20110124,[락]
10498,701978,Meant To Be,[Rachel Belman],20101011,"[봄, 힐링, 카페]"


### 태그와 목록 존재
- song_recommend_test 함수로 변경 (태그와 장르 교집합 비교 -> 장르에서 추출 -> 태그에서 추출 순서로 우선순위 부여)

In [99]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend_test(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 힐링, 잔잔한]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[감성적인, 휴가, 주말, 분위기, 휴식, 책읽을때, 추억, 혼자, 잔잔한]"
4912,229622,Say Hello,[Rosie Thomas],20061212,"[사랑, 겨울, 설렘]"
9,205238,"Dingue, Dingue, Dingue",[Christophe Mae],20110124,[락]
10498,701978,Meant To Be,[Rachel Belman],20101011,"[봄, 힐링, 카페]"
16038,513731,The Sun The Trees (Acoustic Ver.),[Russian Red],20111018,[팝]
16037,489449,Lego House (Acoustic),[Ed Sheeran],20111111,[팝]
9997,392798,Fuerteventura,[Russian Red],20111018,"[나만의Best3, 피쉬슈즈, indie, 인디]"
0,525514,Hey Little Girl,[The Sol],20130506,[락]
9450,598147,Take Your Time,[Louis Yoelin],20131105,"[기분전환, 휴식, 힐링]"


### 노래 목록만 존재

In [26]:
my_tags2 = []
my_songs2 = [525514, 129701, 229622]
rec2 = song_recommend(my_tags2, my_songs2, train_data_sample2, song_tag_appended, gnr_sim)
rec2[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

                                             tags  song_id  \
0                                   [휴식, 힐링, 잔잔한]   191430   
1      [감성적인, 휴가, 주말, 분위기, 휴식, 책읽을때, 추억, 혼자, 잔잔한]   463782   
2                                    [사랑, 겨울, 설렘]   229622   
3                                             [락]   205238   
4                                     [봄, 힐링, 카페]   701978   
..                                            ...      ...   
95                                 [매장음악, 팝, 잔잔한]   315691   
96                                   [드라이브, 스트레스]   458897   
97                                     [스트레스, 운동]   486784   
98                                   [드라이브, 스트레스]   231210   
99  [행복, 집, 꿈, 산책, 매장음악, 드라이브, 바람, 몽환, 가을감성, 잔잔한]   603631   

                                         gnr  issue_date  album_id  \
0           [GN1402, GN1401, GN0901, GN0902]    20140407   2249887   
1           [GN1402, GN1401, GN0901, GN0902]    20170227  10041704   
2           [GN1402, GN1401, GN0901, GN0902] 

,song_id,song_name,artist_name_basket,issue_date,tags
11005,191430,Sing,[Ed Sheeran],20140407,"[휴식, 힐링, 잔잔한]"
15445,463782,Dreamin` Slow,[Mac Demarco],20170227,"[감성적인, 휴가, 주말, 분위기, 휴식, 책읽을때, 추억, 혼자, 잔잔한]"
4912,229622,Say Hello,[Rosie Thomas],20061212,"[사랑, 겨울, 설렘]"
9,205238,"Dingue, Dingue, Dingue",[Christophe Mae],20110124,[락]
10498,701978,Meant To Be,[Rachel Belman],20101011,"[봄, 힐링, 카페]"
16038,513731,The Sun The Trees (Acoustic Ver.),[Russian Red],20111018,[팝]
16037,489449,Lego House (Acoustic),[Ed Sheeran],20111111,[팝]
9997,392798,Fuerteventura,[Russian Red],20111018,"[나만의Best3, 피쉬슈즈, indie, 인디]"
0,525514,Hey Little Girl,[The Sol],20130506,[락]
9450,598147,Take Your Time,[Louis Yoelin],20131105,"[기분전환, 휴식, 힐링]"


### 태그만 존재

In [27]:
my_tags3 = ['락']
my_songs3 = []
rec3 = song_recommend(my_tags3, my_songs3, train_data_sample2, song_tag_appended, gnr_sim)
rec3[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
697,146989,YOUTH,[Troye Sivan],20160123,"[드라이브, 발라드, 느낌있는, 팝송, 그냥, Pop, 잔잔한, 여행, 나만알고싶은..."
698,430106,Everglow,[Coldplay],20151204,"[나만알고싶은, Coldplay, Greenday, 기분전환, 발라드, 질리지않는,..."
706,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407,"[드라이브, 느낌있는, 팝송, 잔잔한, Pop, 여행, 나만알고싶은, 띵곡, 기분전..."
2112,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130,"[귀르가즘, 이별, 달달, 알앤비, 새벽, 기분전환, 우울, 꿀, 발라드, Pop,..."
2114,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228,"[이별, 감성발라드, 혼자있을때, 알앤비, 새벽, 기분전환, 우울, 발라드, 인생곡..."
3503,246531,"모든 날, 모든 순간 (Every day, Every Moment)",[폴킴],20180320,"[비, 버스, 잔잔한, 비_오는날, 여행, 우울, 여름방학, 위로, 그루브, 카페,..."
3506,450838,Slow,[SOLE (쏠)],20180713,"[외로움, 그리움, 아쉬움, 새벽, 버스, 늦잠, 밤, 불면증, 혼자, 겨울밤, 카..."
1872,460823,Paris In The Rain,[Lauv],20181008,"[감성발라드, 오후, 드라이브, 주말, 흥겨운, 팝송, 그냥, 힘든날, 센치한, 잔..."
5349,543371,퇴근버스,[이준호],20140205,"[감성발라드, 주말, 발라드, 감미로운, 느낌있는, 야경, 잔잔한, 여행, 기분전환..."
9839,116114,나의 사춘기에게,[볼빨간사춘기],20170928,"[슬픔, 힘내, 추천곡, 새벽, 우울, 좋은노래, 위로, 힐링, 겨울밤, 감성, 그..."


### 둘 다 없음

In [28]:
my_tags4 = []
my_songs4 = []
rec4 = song_recommend(my_tags4, my_songs4, train_data_sample2, song_tag_appended, gnr_sim)
rec4[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]

,song_id,song_name,artist_name_basket,issue_date,tags
89,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115,"[겨울노래, 따듯한, 사무실, 캐럴, 동요, 크리스마스송, 연말, 아이돌, 눈오는날..."
119,457519,꿀차,[우효],20180102,"[겨울노래, 따듯한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기, 겨울왕국, 크..."
120,453762,너 정말 예쁘다,[최낙타],20180410,"[겨울노래, 따듯한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기, 겨울왕국, 크..."
121,349398,LOVE YA!,[혁오 (HYUKOH)],20180531,"[겨울노래, 따듯한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기, 겨울왕국, 크..."
122,631142,편지,[장희원],20180603,"[겨울노래, 봄, 따듯한, 따뜻한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기,..."
123,406082,하늘엔 별이 떠있고 너만큼은 빛나질 않아,[이민혁],20180903,"[겨울노래, 따듯한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기, 겨울왕국, 크..."
124,548389,사랑에 빠졌네,[정준일],20181101,"[겨울노래, 따듯한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기, 겨울왕국, 크..."
125,205179,꿀맛,[정미애],20190815,"[겨울노래, 크리스마스송, 드라이브, 매장, 필라테스, 따듯한, 운동, 여행, 캐럴..."
126,567076,숨겨진 세상 (Into the Unknown End Credit Version) (...,[태연 (TAEYEON)],20191115,"[겨울노래, 따듯한, 캐럴, 크리스마스송, 연말, 눈오는날, 분위기, 겨울왕국, 크..."
180,418694,Attention,[Charlie Puth],20180511,"[드라이브, 일렉, 2017, 트로피컬하우스, 잔잔한, Pop, 운동, 여행, 나만..."


# 모델 평가
- [참고](https://chrisjune-13837.medium.com/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80%EB%B0%A9%EB%B2%95-with-python-9932097f0ff9)

# Recall@K

In [83]:
def get_recall_k(y_true, y_pred):
    recall_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(true_items) if len(true_items) > 0 else 0
    return recall

my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend_test(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
pred_list = rec1['song_id'].tolist()

recall_k = get_recall_k(my_songs1, pred_list)
print("Recall@K (K=10): {:.2f}".format(recall_k))

Recall@K (K=10): 0.67


# Precision@K

In [84]:
def get_precision_k(y_true, y_pred):
    precision_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(pred_items) if len(pred_items) > 0 else 0
    return recall

my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend_test(my_tags1, my_songs1, train_data_sample2, song_tag_appended, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date', 'tags']]
pred_list = rec1['song_id'].tolist()

precision_k = get_precision_k(my_songs1, pred_list)
print("Precision@K (K=10): {:.2f}".format(precision_k))

Precision@K (K=10): 0.20


# Average Precision@K
- 아이템 개수가 매우 적은 경우, 모델의 성능 또한 낮아지는 문제가 발생
    - 해결방법
        1. Thresholding : 사용자 플레이리스트 내 음악이 일정 개수 이상인 경우만 적용
        2. Item Weighting : hits 에 1이 아닌 가중치를 이용. 이 때 precision 이 1 보다 크지 않게 하기 위해, 정규화 필요

In [31]:
def get_ap_k(y_true, y_pred, k):
    pred_items = y_pred[:k]
    hits = []
    for item in pred_items:
        if item in y_true:
            hits.append(1)
        else:
            hits.append(0)
    precision_values = []
    for i in range(1, k+1):
        precision_values.append(sum(hits[:i]) / i)
        
    print(precision_values)
    
    if len(precision_values) == 0:
        return 0
    else:
        return sum(precision_values) / len(precision_values)
    
ap_k = get_ap_k(my_songs1, pred_list, 10)
print("AP@K (K=10): {:.2f}".format(ap_k))

[0.0, 0.0, 0.3333333333333333, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.125, 0.2222222222222222, 0.2]
AP@K (K=10): 0.16


# Mean Average Precision@K

In [85]:
def get_map_k(y_true, y_pred, k):
    sum_ap = 0
    for true_item, pred_item in zip(y_true, y_pred):
        ap_k = get_ap_k(true_item, pred_item, k)
        sum_ap += ap_k
    if len(y_true) == 0:
        return 0
    else:
        return sum_ap / len(y_true)
    
all_my_songs = []
all_pred_songs = []
all_my_songs.append(my_songs1)
all_pred_songs.append(pred_list)

map_k = get_map_k(all_my_songs, all_pred_songs, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[0.0, 0.0, 0.3333333333333333, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.125, 0.2222222222222222, 0.2]
MAP@K (K=10): 0.16


In [129]:
test_data_sample = train_data[:500]
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend_test(tags, plist, train_data_sample2, song_tag_appended, gnr_sim)
    pred_list.append(recommended['song_id'].tolist())
#print(pred_list[0])


In [131]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.3333333333333333, 0.5, 0.4, 0.3333333333333333, 0.2857142857142857, 0.25, 0.2222222222222222, 0.2]
[0.0, 0.0, 0.0, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.125, 0.1111111111111111, 0.1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.5, 0.6666666666666666, 0.75, 0.6, 0.6666666666666666, 0.7142857142857143, 0.75, 0.7777777777777778, 0.8]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.5, 0.6666666666666666, 0.5, 0.4, 0.3333333333333333, 0.42857142857142855, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[